In [77]:
import requests
import pandas as pd 
import os
from dotenv import load_dotenv

In [28]:
load_dotenv()

API_KEY1 = os.getenv("API_KEY1")
API_KEY2 = os.getenv("API_KEY2")
API_KEY3 = os.getenv("API_KEY3")
API_KEY4 = os.getenv("API_KEY4")
API_KEY5 = os.getenv("API_KEY5")

# URL_NUTRIENTS = "https://api.spoonacular.com/recipes/findByNutrients"
# URL_OTHERS = "https://api.spoonacular.com/recipes/complexSearch"
URL_PLANNER = "https://api.spoonacular.com/mealplanner/generate"

MAX_REQUEST_PER_RESULT = 100

### **1. Primera llamada: API GENERATE MEAL PLANNER**

In [42]:
def filter_id_by_meal(data_in_json) -> dict[str, list[int]]:
    raw_meals = data_in_json["week"]
    days_and_meals = {}
    for day, meals in raw_meals.items():
        meals_id_list = []
        daily_meals = meals["meals"]
        for meal in daily_meals:
            meals_id_list.append(meal["id"])
            days_and_meals[day] = meals_id_list
    return days_and_meals

In [43]:
def meal_planner(
    time: str = 'Week', 
    calories: float = 1800, 
    diet: str = 'vegetarian', 
    exlude: str = None
) -> dict[str, list[int]]:
    """
    Planificador de comidas.
    
    Args:
        time: tiempo para planificar (week, day)
        calories: calorías diarias recomendadas
        diet: tipo de dieta (vegan, vegetarian, gluten free, ...)
    
    Returns:
        meal_plan: plan de comidas
    """
    
    params = {
        "apiKey": API_KEY1,
        "timeFrame": time,#.lower(),
        "targetCalories": calories,
        # "diet": diet.lower(),
        # "exclude": exlude
    }
    
    response = requests.get(URL_PLANNER, params=params)
    if response.status_code == 200:
        data = response.json()
        return filter_id_by_meal(data)
    else:
        print(f"Error en la solicitud: {response.status_code} - {response.text}")
        return {}

In [49]:
days_and_ids = meal_planner()
days_and_ids

{'monday': [660368, 1171085, 659834],
 'tuesday': [918034, 1095806, 652990],
 'wednesday': [659604, 1697577, 650523],
 'thursday': [649300, 652417, 648983],
 'friday': [659081, 643789, 637808],
 'saturday': [662581, 715769, 661689],
 'sunday': [662291, 1697749, 650235]}

### 2. Segunda llamada: API RECIPE INFORMATION

In [50]:
column_names = [
    'Meal', 
    'Name', 
    'Ingredients', 
    'Instructions', 
    'Calories', 
    'Carbs',
    'Fat', 
    'Protein'
]

In [75]:
def filter_name_and_ingredients(recipe_details) -> dict[str, str]:
    
    def filter_ingredients() -> str:
        extended_ingredients: list[dict[str, str]] = recipe_details['extendedIngredients']
        ingredients_list = [
            ingredient['original'] for ingredient in extended_ingredients
            ]
        ingredients_str = ', '.join(ingredients_list)
        return ingredients_str
    
    name = recipe_details['title']
    ingredients = filter_ingredients()
    
    info_filtered = {'Name': name, 'Ingredients': ingredients}
    return info_filtered

def filter_instructions(recipe_details) -> dict[str, str]:
    steps: list[dict[str, str]] = recipe_details[0]["steps"]
    instructions_list = [
        f"{i + 1}. {instruction["step"]}" for i, instruction in enumerate(steps) 
    ]
    instructions_str = ' '.join(instructions_list)
    instructions_filtered = {'Instructions': instructions_str}
    return instructions_filtered

def filter_nutrition_facts(recipe_details) -> dict[str, int]:
    nutrition_facts_filtered = {
        'Calories': recipe_details['calories'],
        'Carbs': recipe_details['carbs'],
        'Fat': recipe_details['fat'],
        'Protein': recipe_details['protein']
    }
    return nutrition_facts_filtered

In [53]:
daily_meals_plan = {
        "Meal": [],
        "Name": [],
        "Ingredients": [],
        "Instructions": [],
        "Calories": [],
        "Carbs": [],
        "Fat": [],
        "Protein": [],
        }

In [54]:
def set_recipe_attributes(
    daily_meals_plan: dict[str, list], 
    data_to_insert: dict[str, str]
    ):
    for atribute_name, atribute_value in data_to_insert.items():
        daily_meals_plan[atribute_name].append(atribute_value)

In [55]:
def get_recipe_name_and_ingredients(
    days_and_ids: dict[str, list[int]],
    daily_meals_plan: dict[str, list[str]]
    ) -> dict[str, 
              dict[str, str | list[str]]
            ]:
    """ 
    Example days_and_ids:
    {
     'monday': [641700, 642593, 640136],
     'tuesday': [640767, 1697541, 643642],
     'wednesday': [647555, 642593, 641166],
     'thursday': [653775, 650127, 634891],
     'friday': [715569, 1165787, 638248],
     'saturday': [632639, 1502521, 648983],
     'sunday': [641185, 1697541, 654681]
    }
    
    Example daily_meals_plan:
    {
     "Meal": [monday_1, monday_2, monday_3],
     "Name": [recipe_1, recipe_2, recipe_3],
     "Ingredients": [--------, --------, --------],
     "Instructions": [--------, --------, --------],
     "Calories": [643, 213, 4354],
     "Carbs": [23, 78, 234],
     "Fat": [354, 235, 56],
     "Protein": [132, 346, 586],
    }
    """
    for day, meal_ids_list in days_and_ids.items():
        
        for num_meal, meal_id in enumerate(meal_ids_list):
            
            url_with_id = f"https://api.spoonacular.com/recipes/{meal_id}/information"
            params = {
                "apiKey": API_KEY1
            }
            
            response = requests.get(url_with_id, params=params)
            
            if response.status_code == 200:
                data = response.json()
                
                filtered_name_ingredients = filter_name_and_ingredients(data)
                set_recipe_attributes(daily_meals_plan, filtered_name_ingredients)
                
                day_and_number = f"{day} {num_meal + 1}"
                daily_meals_plan["Meal"].append(day_and_number)
            else:
                print(f"Error en la solicitud: {response.status_code} - {response.text}")
    return daily_meals_plan

In [59]:
def get_instructions(
    days_and_ids: dict[str, list[int]],
    daily_meals_plan: dict[str, list[str]]
    ) -> dict[str, 
              dict[str, str | list[str]]
            ]:
    """ 
    Example days_and_ids:
    {
     'monday': [641700, 642593, 640136],
     'tuesday': [640767, 1697541, 643642],
     'wednesday': [647555, 642593, 641166],
     'thursday': [653775, 650127, 634891],
     'friday': [715569, 1165787, 638248],
     'saturday': [632639, 1502521, 648983],
     'sunday': [641185, 1697541, 654681]
    }
    
    Example daily_meals_plan:
    {
     "Meal": [monday_1, monday_2, monday_3],
     "Name": [recipe_1, recipe_2, recipe_3],
     "Ingredients": [--------, --------, --------],
     "Instructions": [--------, --------, --------],
     "Calories": [643, 213, 4354],
     "Carbs": [23, 78, 234],
     "Fat": [354, 235, 56],
     "Protein": [132, 346, 586],
    }
    """
    for day, meal_ids_list in days_and_ids.items():
        
        for meal_id in meal_ids_list:
            
            url_with_id = f"https://api.spoonacular.com/recipes/{meal_id}/analyzedInstructions"
            params = {
                "apiKey": API_KEY1
            }
            
            response = requests.get(url_with_id, params=params)
            
            if response.status_code == 200:
                data = response.json()
                
                filtered_instructions = filter_instructions(data)
                set_recipe_attributes(daily_meals_plan, filtered_instructions)
            else:
                print(f"Error en la solicitud: {response.status_code} - {response.text}")
    return daily_meals_plan

In [73]:
def get_nutrition_facts(
    days_and_ids: dict[str, list[int]],
    daily_meals_plan: dict[str, list[str]]
    ) -> dict[str, 
              dict[str, str | list[str]]
            ]:
    """ 
    Example days_and_ids:
    {
     'monday': [641700, 642593, 640136],
     'tuesday': [640767, 1697541, 643642],
     'wednesday': [647555, 642593, 641166],
     'thursday': [653775, 650127, 634891],
     'friday': [715569, 1165787, 638248],
     'saturday': [632639, 1502521, 648983],
     'sunday': [641185, 1697541, 654681]
    }
    
    Example daily_meals_plan:
    {
     "Meal": [monday_1, monday_2, monday_3],
     "Name": [recipe_1, recipe_2, recipe_3],
     "Ingredients": [--------, --------, --------],
     "Instructions": [--------, --------, --------],
     "Calories": [643, 213, 4354],
     "Carbs": [23, 78, 234],
     "Fat": [354, 235, 56],
     "Protein": [132, 346, 586],
    }
    """
    for day, meal_ids_list in days_and_ids.items():
        
        for meal_id in meal_ids_list:
            
            url_with_id = f"https://api.spoonacular.com/recipes/{meal_id}/nutritionWidget.json"
            params = {
                "apiKey": API_KEY1
            }
            
            response = requests.get(url_with_id, params=params)
            
            if response.status_code == 200:
                data = response.json()
                
                filtered_nutrition_facts = filter_nutrition_facts(data)
                set_recipe_attributes(daily_meals_plan, filtered_nutrition_facts)
            else:
                print(f"Error en la solicitud: {response.status_code} - {response.text}")
    return daily_meals_plan

In [76]:
prueba_nutricion = get_nutrition_facts(days_and_ids, daily_meals_plan)
prueba_nutricion

{'Meal': ['monday 1',
  'monday 2',
  'monday 3',
  'tuesday 1',
  'tuesday 2',
  'tuesday 3',
  'wednesday 1',
  'wednesday 2',
  'wednesday 3',
  'thursday 1',
  'thursday 2',
  'thursday 3',
  'friday 1',
  'friday 2',
  'friday 3',
  'saturday 1',
  'saturday 2',
  'saturday 3',
  'sunday 1',
  'sunday 2',
  'sunday 3'],
 'Name': ['Smoked Salmon Eggs Benedict With Lemon Dill Hollandaise',
  'How to Make an Easy Chicken Enchilada',
  'Sheesh Kabobs with Yogurt Dip',
  'Pumpkin Doughnuts',
  'Spanish style salmon fillets',
  'Nearly Famous Chicken Tortilla Soup',
  'Scrambled tofu with rocket',
  'Spanish Sardines Pasta',
  'Macadamia Crusted Whitefish',
  'Latkes; Fried Vegetable Pancakes from Europe and the Middle East',
  'Moroccan chickpea and lentil stew',
  'Knishes - Potato Filling',
  'Salmon Frittata',
  'Fried Salmon Cakes',
  'Chestnut Pappardelle',
  'Sweet Potato Hashbrown Breakfast Bake',
  'Broccolini Quinoa Pilaf',
  'Stir-Fried Shredded Chicken and Mushrooms With Bal

In [71]:
prueba_instructions = get_instructions(days_and_ids, daily_meals_plan)
prueba_instructions

{'Meal': ['monday 1',
  'monday 2',
  'monday 3',
  'tuesday 1',
  'tuesday 2',
  'tuesday 3',
  'wednesday 1',
  'wednesday 2',
  'wednesday 3',
  'thursday 1',
  'thursday 2',
  'thursday 3',
  'friday 1',
  'friday 2',
  'friday 3',
  'saturday 1',
  'saturday 2',
  'saturday 3',
  'sunday 1',
  'sunday 2',
  'sunday 3'],
 'Name': ['Smoked Salmon Eggs Benedict With Lemon Dill Hollandaise',
  'How to Make an Easy Chicken Enchilada',
  'Sheesh Kabobs with Yogurt Dip',
  'Pumpkin Doughnuts',
  'Spanish style salmon fillets',
  'Nearly Famous Chicken Tortilla Soup',
  'Scrambled tofu with rocket',
  'Spanish Sardines Pasta',
  'Macadamia Crusted Whitefish',
  'Latkes; Fried Vegetable Pancakes from Europe and the Middle East',
  'Moroccan chickpea and lentil stew',
  'Knishes - Potato Filling',
  'Salmon Frittata',
  'Fried Salmon Cakes',
  'Chestnut Pappardelle',
  'Sweet Potato Hashbrown Breakfast Bake',
  'Broccolini Quinoa Pilaf',
  'Stir-Fried Shredded Chicken and Mushrooms With Bal

In [56]:
prueba = get_recipe_name_and_ingredients(days_and_ids, daily_meals_plan)
prueba

{'Meal': ['monday 1',
  'monday 2',
  'monday 3',
  'tuesday 1',
  'tuesday 2',
  'tuesday 3',
  'wednesday 1',
  'wednesday 2',
  'wednesday 3',
  'thursday 1',
  'thursday 2',
  'thursday 3',
  'friday 1',
  'friday 2',
  'friday 3',
  'saturday 1',
  'saturday 2',
  'saturday 3',
  'sunday 1',
  'sunday 2',
  'sunday 3'],
 'Name': ['Smoked Salmon Eggs Benedict With Lemon Dill Hollandaise',
  'How to Make an Easy Chicken Enchilada',
  'Sheesh Kabobs with Yogurt Dip',
  'Pumpkin Doughnuts',
  'Spanish style salmon fillets',
  'Nearly Famous Chicken Tortilla Soup',
  'Scrambled tofu with rocket',
  'Spanish Sardines Pasta',
  'Macadamia Crusted Whitefish',
  'Latkes; Fried Vegetable Pancakes from Europe and the Middle East',
  'Moroccan chickpea and lentil stew',
  'Knishes - Potato Filling',
  'Salmon Frittata',
  'Fried Salmon Cakes',
  'Chestnut Pappardelle',
  'Sweet Potato Hashbrown Breakfast Bake',
  'Broccolini Quinoa Pilaf',
  'Stir-Fried Shredded Chicken and Mushrooms With Bal

In [79]:
daily_meals_plan_example = pd.DataFrame(daily_meals_plan)
daily_meals_plan_example.to_csv("daily_meals_plan_example.csv", index=False)
daily_meals_plan_example.to_json("daily_meals_plan_example.json", orient="records")